In [161]:
import pandas as pd
import datetime
import re

In [162]:
up_keywords = []
with open("看漲關鍵字.txt",'r',encoding = 'UTF8') as file:
    for data in file.readlines():
        data = data.strip()
        up_keywords.append(data)

In [163]:
down_keywords = []
with open("看跌關鍵字.txt",'r',encoding = 'UTF8') as file:
    for data in file.readlines():
        data = data.strip()
        down_keywords.append(data)

In [164]:
word_vec = up_keywords + down_keywords

In [165]:
size = len(word_vec)

In [166]:
size

1400

In [167]:
up_news = pd.read_csv('看漲-幅度大於2%.csv')
down_news = pd.read_csv('看跌-幅度大於2%.csv')

In [168]:
up_news['year'] = pd.DatetimeIndex(up_news['normalized_time']).year
up_news['month'] = pd.DatetimeIndex(up_news['normalized_time']).month

In [169]:
# up_news

In [170]:
down_news['year'] = pd.DatetimeIndex(down_news['normalized_time']).year
down_news['month'] = pd.DatetimeIndex(down_news['normalized_time']).month

In [171]:
# down_news

In [172]:
size_dict = {2016: {}, 2017: {}, 2018: {}, 2019: {}, 2020: {}}


for year in range(2016, 2021):
    for month in range(1, 13):
        up = up_news.loc[(up_news['year'] == year) & (up_news['month'] == month)]
        down = down_news.loc[(down_news['year'] == year) & (down_news['month'] == month)]
        size = len(up) + len(down)
        size_dict[year][month] = size
    

In [173]:
pd.DataFrame(size_dict)

,2016,2017,2018,2019,2020
1,3,69,16,78,91
2,0,99,1,74,0
3,8,58,18,114,39
4,2,61,42,114,19
5,5,12,39,149,13
6,14,30,108,65,42
7,2,14,219,79,55
8,0,97,59,60,22
9,8,140,137,48,19
10,0,48,107,34,11


In [174]:
size_1719 = []

for year in range(2017, 2020):
    for month in range(1, 13):
        up = up_news.loc[(up_news['year'] == year) & (up_news['month'] == month)]
        down = down_news.loc[(down_news['year'] == year) & (down_news['month'] == month)]
        size = len(up) + len(down)

        size_1719.append([year, month, size])
    

In [175]:
size_1719

[[2017, 1, 69],
 [2017, 2, 99],
 [2017, 3, 58],
 [2017, 4, 61],
 [2017, 5, 12],
 [2017, 6, 30],
 [2017, 7, 14],
 [2017, 8, 97],
 [2017, 9, 140],
 [2017, 10, 48],
 [2017, 11, 30],
 [2017, 12, 30],
 [2018, 1, 16],
 [2018, 2, 1],
 [2018, 3, 18],
 [2018, 4, 42],
 [2018, 5, 39],
 [2018, 6, 108],
 [2018, 7, 219],
 [2018, 8, 59],
 [2018, 9, 137],
 [2018, 10, 107],
 [2018, 11, 124],
 [2018, 12, 64],
 [2019, 1, 78],
 [2019, 2, 74],
 [2019, 3, 114],
 [2019, 4, 114],
 [2019, 5, 149],
 [2019, 6, 65],
 [2019, 7, 79],
 [2019, 8, 60],
 [2019, 9, 48],
 [2019, 10, 34],
 [2019, 11, 46],
 [2019, 12, 194]]

In [176]:
total = len(size_1719)
train_size = 5
iteration = total - train_size

In [179]:
def preprocess(sent): # 不切 ngram
    sent = re.sub(r'[^\w]',"",sent)
    sent = re.sub(r'[A-Za-z0-9]',"",sent)
    return sent

In [181]:
stopwords = []
with open("stopwords.TXT",'r',encoding = 'UTF8') as file:
    for data in file.readlines():
        data = data.strip()
        stopwords.append(data)

In [204]:
def get_news_vec(news, y):
    news_vec = []

    for index, row in news.iterrows():
        text_vec = []
        content = preprocess(row["title"] + row["content"])

        data = {
            'index': index,
            'date': row['normalized_time'],
            'year': row['year'],
            'month': row['month'],
            'title': row['title'],
            'content': row['content'],
            'y': y
        }

        for word in word_vec:
            data[word] = 0
            if word in content:
                data[word] = 1

        news_vec.append(data)
    return news_vec

In [235]:
def export_data(data, file_name, file_type):
    base_data = data[data.columns[:7]]
    base_data.to_csv(r'基本資料_' + file_type + '_' + file_name + '.csv', index = False)
    n = 0 - len(word_vec)
    X = data[data.columns[n:]]
    X.to_csv(r'X_' + file_type + '_' +  file_name + '.csv', index = False)
    data['y'].to_csv(r'Y_' + file_type + '_' + file_name + '.csv', index = False)

In [214]:
# list(up_news.loc[(up_news['year'] == year) & (up_news['month'] == month)]['content'])
# word_vec

In [236]:
duration_data = []

for i in range(iteration):
    up = pd.concat([
        up_news.loc[(up_news['year'] == size_1719[i][0]) & (up_news['month'] == size_1719[i][1])],
        up_news.loc[(up_news['year'] == size_1719[i+1][0]) & (up_news['month'] == size_1719[i+1][1])],
        up_news.loc[(up_news['year'] == size_1719[i+2][0]) & (up_news['month'] == size_1719[i+2][1])],
        up_news.loc[(up_news['year'] == size_1719[i+3][0]) & (up_news['month'] == size_1719[i+3][1])],
        up_news.loc[(up_news['year'] == size_1719[i+4][0]) & (up_news['month'] == size_1719[i+4][1])],
    ],axis = 0)
    
    down = pd.concat([
        down_news.loc[(down_news['year'] == size_1719[i][0]) & (down_news['month'] == size_1719[i][1])],
        down_news.loc[(down_news['year'] == size_1719[i+1][0]) & (down_news['month'] == size_1719[i+1][1])],
        down_news.loc[(down_news['year'] == size_1719[i+2][0]) & (down_news['month'] == size_1719[i+2][1])],
        down_news.loc[(down_news['year'] == size_1719[i+3][0]) & (down_news['month'] == size_1719[i+3][1])],
        down_news.loc[(down_news['year'] == size_1719[i+4][0]) & (down_news['month'] == size_1719[i+4][1])],
    ],axis = 0)
    
    up_test = up_news.loc[(up_news['year'] == size_1719[i+5][0]) & (up_news['month'] == size_1719[i+5][1])]
    down_test = down_news.loc[(down_news['year'] == size_1719[i+5][0]) & (down_news['month'] == size_1719[i+5][1])]
    
    news_vec = get_news_vec(up, 1)
    news_vec += get_news_vec(down, -1)

    news_vec_test = get_news_vec(up_test, 1)
    news_vec_test += get_news_vec(down_test, -1)
    
    train_data = pd.DataFrame(news_vec)
    test_data = pd.DataFrame(news_vec_test)

    duration = str(size_1719[i][0]) + '.' + str(size_1719[i][1]) + '~' + str(size_1719[i+4][0]) + '.' + str(size_1719[i+4][1])
    export_data(train_data, str(i), 'training')
    export_data(test_data, str(i), 'testing')
    duration_data.append({
        'file_index': i,
        'training_duration': duration,
        'testing_duration': str(size_1719[i+5][0]) + '.' + str(size_1719[i+5][1]),
        'training_size': len(train_data),
        'testing_size': len(test_data)
    })

In [234]:
duration_pd = pd.DataFrame(duration_data)
duration_pd.to_csv(r'檔名及區間紀錄.csv', index = False)

In [241]:
duration_data = []

for i in range(iteration-1):
    up = pd.concat([
        up_news.loc[(up_news['year'] == size_1719[i][0]) & (up_news['month'] == size_1719[i][1])],
        up_news.loc[(up_news['year'] == size_1719[i+1][0]) & (up_news['month'] == size_1719[i+1][1])],
        up_news.loc[(up_news['year'] == size_1719[i+2][0]) & (up_news['month'] == size_1719[i+2][1])],
        up_news.loc[(up_news['year'] == size_1719[i+3][0]) & (up_news['month'] == size_1719[i+3][1])],
        up_news.loc[(up_news['year'] == size_1719[i+4][0]) & (up_news['month'] == size_1719[i+4][1])],
    ],axis = 0)
    
    down = pd.concat([
        down_news.loc[(down_news['year'] == size_1719[i][0]) & (down_news['month'] == size_1719[i][1])],
        down_news.loc[(down_news['year'] == size_1719[i+1][0]) & (down_news['month'] == size_1719[i+1][1])],
        down_news.loc[(down_news['year'] == size_1719[i+2][0]) & (down_news['month'] == size_1719[i+2][1])],
        down_news.loc[(down_news['year'] == size_1719[i+3][0]) & (down_news['month'] == size_1719[i+3][1])],
        down_news.loc[(down_news['year'] == size_1719[i+4][0]) & (down_news['month'] == size_1719[i+4][1])],
    ],axis = 0)
    
    up_test = pd.concat([
        up_news.loc[(up_news['year'] == size_1719[i+5][0]) & (up_news['month'] == size_1719[i+5][1])],
        up_news.loc[(up_news['year'] == size_1719[i+6][0]) & (up_news['month'] == size_1719[i+6][1])],
    ],axis = 0)
    down_test = pd.concat([
        down_news.loc[(down_news['year'] == size_1719[i+5][0]) & (down_news['month'] == size_1719[i+5][1])],
        down_news.loc[(down_news['year'] == size_1719[i+6][0]) & (down_news['month'] == size_1719[i+6][1])],
    ],axis = 0)
    
    news_vec = get_news_vec(up, 1)
    news_vec += get_news_vec(down, -1)

    news_vec_test = get_news_vec(up_test, 1)
    news_vec_test += get_news_vec(down_test, -1)
    
    train_data = pd.DataFrame(news_vec)
    test_data = pd.DataFrame(news_vec_test)

    duration = str(size_1719[i][0]) + '.' + str(size_1719[i][1]) + '~' + str(size_1719[i+4][0]) + '.' + str(size_1719[i+4][1])
    export_data(train_data, str(i), 'training')
    export_data(test_data, str(i), 'testing')
    duration_data.append({
        'file_index': i,
        'training_duration': duration,
        'testing_duration': str(size_1719[i+5][0]) + '.' + str(size_1719[i+5][1]) + '~' + str(size_1719[i+6][0]) + '.' + str(size_1719[i+6][1]),
        'training_size': len(train_data),
        'testing_size': len(test_data)
    })

In [242]:
duration_data

[{'file_index': 0,
  'training_duration': '2017.1~2017.5',
  'testing_duration': '2017.6~2017.7',
  'training_size': 299,
  'testing_size': 44},
 {'file_index': 1,
  'training_duration': '2017.2~2017.6',
  'testing_duration': '2017.7~2017.8',
  'training_size': 260,
  'testing_size': 111},
 {'file_index': 2,
  'training_duration': '2017.3~2017.7',
  'testing_duration': '2017.8~2017.9',
  'training_size': 175,
  'testing_size': 237},
 {'file_index': 3,
  'training_duration': '2017.4~2017.8',
  'testing_duration': '2017.9~2017.10',
  'training_size': 214,
  'testing_size': 188},
 {'file_index': 4,
  'training_duration': '2017.5~2017.9',
  'testing_duration': '2017.10~2017.11',
  'training_size': 293,
  'testing_size': 78},
 {'file_index': 5,
  'training_duration': '2017.6~2017.10',
  'testing_duration': '2017.11~2017.12',
  'training_size': 329,
  'testing_size': 60},
 {'file_index': 6,
  'training_duration': '2017.7~2017.11',
  'testing_duration': '2017.12~2018.1',
  'training_size': 32

In [243]:
duration_pd = pd.DataFrame(duration_data)
duration_pd.to_csv(r'檔名及區間紀錄.csv', index = False)